# Advanced Lane Lines

---

## Module Imports

---

In [1]:
import cv2
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import mean_squared_error

## Camera Calibration

---

In [2]:
def calibrate_camera():
    
    obj_point_container = []
    img_point_container = []
    
    # The number of corners
    nx = 9
    ny = 6

    # Import images 
    images = glob.glob('camera_cal/calibration*.jpg')

    # Preparing the object's corners
    obj_point = np.zeros((ny*nx,3), np.float32)
    obj_point[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    for index, file_name in enumerate(images):
        image = cv2.imread(file_name)
        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray_image, (nx, ny), None)
        # If found, draw corners
        if ret == True:
            # draw and display the corners
            cv2.drawChessboardCorners(image, (nx, ny), corners, ret)
            obj_point_container.append(obj_point)
            img_point_container.append(corners)
            
    return (obj_point_container, img_point_container)



## Undistort Camera

---

In [ ]:
def undistort_camera(img_points, obj_points, img):
    img_size = (img.shape[1], img.shape[0])
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, img_size,None,None)
    return cv2.undistort(img, mtx, dist, None, mtx)